In [ ]:
# https://simpleelastix.readthedocs.io/GroupwiseRegistration.html
# https://github.com/SuperElastix/SimpleElastix/issues/23

In [5]:
import SimpleITK as sitk
"""
We may need to change the image origins to make sure they are the same
. This can be done by copying the origin of one of the images origin = firstImage.GetOrigin()
 and setting it to the others otherImages.SetOrigin(origin)
"""


# # Concatenate the ND images into one (N+1)D image
# population = ['image1.hdr', ..., 'imageN.hdr']
# vectorOfImages = sitk.VectorOfImage()

# for filename in population:
#   vectorOfImages.push_back(sitk.ReadImage(filename))

# image = sitk.JoinSeries(vectorOfImages)

# # Register
# elastixImageFilter = sitk.ElastixImageFilter()
# elastixImageFilter.SetFixedImage(image)
# elastixImageFilter.SetMovingImage(image)
# elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap('groupwise'))
# elastixImageFilter.Execute()

# transformParameterMap = elastixImageFilter.GetTransformParameterMap()
# transformixImageFilter = sitk.TransformixImageFilter()
# transformixImageFilter.SetTransformParameterMap(transformParameterMap)

# population = ['image1.hdr', 'image2.hdr', ... , 'imageN.hdr']

# here we need to concatenate labels to 4d tensor


# for filename in population:
#     transformixImageFilter.SetMovingImage(sitk.ReadImage(filename))
#     transformixImageFilter.Execute()
#     sitk.WriteImage(transformixImageFilter.GetResultImage(), "result_"+filename)



# # Turn off interpolation for labels (a bug in SWIG wrapping prevents us from writing directly to a parameter map in a vector so we have unwrap it first) 
# transformParameterMap = transformParameterMapVector[0]
# transformParameterMap["ResampleInterpolator"] = ["FinalNearestNeighborInterpolator"]
# resultLabel = sitk.Transformix(label4d, transformParameterMap)

'\nWe may need to change the image origins to make sure they are the same\n. This can be done by copying the origin of one of the images origin = firstImage.GetOrigin()\n and setting it to the others otherImages.SetOrigin(origin)\n'

In [3]:
import pandas as pd
import SimpleITK as sitk

df = pd.read_csv('/mnt/disks/sdb/metadata/processedMetaData.csv')
row= df.iloc[19]
pathT2w= row['t2w']
pathHbv= row['hbv']
pathADC= row['adc']
patht2wLabel= row['reSampledPath']


node = slicer.cli.runSync(slicer.modules.slicerelastixcli)
print(node)

'/mnt/disks/sdb/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix -f /mnt/disks/sdb/orig/10019/10019_1000019_t2w.mha -m /mnt/disks/sdb/orig/10019/10019_1000019_hbv.mha -out /mnt/disks/sdb/exploration -p /mnt/disks/sdb/piCaiCode/preprocessing/registration/parameters.txt'


In [23]:
from subprocess import Popen
import subprocess
import SimpleITK as sitk
import pandas as pd
import multiprocessing as mp
import functools
from functools import partial
import sys


elacticPath='/mnt/disks/sdb/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix'
reg_prop='/mnt/disks/sdb/piCaiCode/preprocessing/registration/parameters.txt'
def reg_adc_hbv_to_t2w(row,colName):
    """
    registers adc and hbv images to t2w image
    first we need to create directories for the results
    then we suply hbv and adc as moving images and t2w as static one - and register in reference to it
    we do it in multiple threads at once and we waiteach time the process finished
    """
    row=row[1]
    patId=str(row['patient_id'])
    path=str(row[colName])
    if(len(path)>1):
        outPath = path.replace(".mha","_for_"+colName)
        cmd='mkdir '+ outPath
        p = Popen(cmd, shell=True)
        p.wait()
        cmd=f"{elacticPath} -f {row['t2w']} -m {path} -out {outPath} -p {reg_prop}"
        print(cmd)
        try:
            p = Popen(cmd, shell=True)
        except:
            print("error in patId")
        p.wait()

        
for keyWord in ['adc','hbv']:    
    resList=[]
    with mp.Pool(processes = mp.cpu_count()) as pool:
        pool.map(partial(reg_adc_hbv_to_t2w,colName=keyWord)  ,list(df.iterrows()))    
 

/mnt/disks/sdb/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix -f /mnt/disks/sdb/orig/10076/10076_1000076_t2w.mha -m /mnt/disks/sdb/orig/10076/10076_1000076_adc.mha -out /mnt/disks/sdb/orig/10076/10076_1000076_adc_for_adc -p /mnt/disks/sdb/piCaiCode/preprocessing/registration/parameters.txt
/mnt/disks/sdb/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix -f /mnt/disks/sdb/orig/10019/10019_1000019_t2w.mha -m /mnt/disks/sdb/orig/10019/10019_1000019_adc.mha -out /mnt/disks/sdb/orig/10019/10019_1000019_adc_for_adc -p /mnt/disks/sdb/piCaiCode/preprocessing/registration/parameters.txt
/mnt/disks/sdb/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix -f /mnt/disks/sdb/orig/10038/10038_1000038_t2w.mha -m /mnt/disks/sdb/orig/10038/10038_1000038_adc.mha -out /mnt/disks/sdb/orig/10038/10038_1000038_adc_for_adc -p /mnt/disks/sdb/piCaiCode/preprocessing/registration/parameters.txt
/mnt/disks/sdb/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/

Process ForkPoolWorker-56:
Process ForkPoolWorker-57:


KeyboardInterrupt: 